Будем использовать физическими величинами с измерениями. Модуль PhysicalQuantities сломался, не обновлялся два года, поэтому придется попробовать `pint` + `uncertainties`.

In [55]:
from pint import UnitRegistry
from uncertainties import ufloat
from scipy.constants import k as k_J_per_K
import math

In [42]:
H_отсчета = (ufloat(200, 10) * ureg.kilometer).to_base_units()
H_отсчета

<Quantity(200000+/-10000, 'meter')>

In [43]:
P_отсчета =  (ufloat(8.53, 1)* 1e-5 * ureg.pascal).to_base_units()
P_отсчета

<Quantity((8.53000+/-1.00000)e-05, 'kilogram / meter / second ** 2')>

In [44]:
H = (ufloat(60, 10) * ureg.kilometer).to_base_units()
H

<Quantity(60000.0+/-10000.0, 'meter')>

In [45]:
σ = (ufloat(8, 3) * ureg.angstrom * ureg.angstrom).to_base_units()
σ

<Quantity((8.00000+/-3.00000)e-20, 'meter ** 2')>

In [46]:
T_отсчета = (ufloat(854.4, 200) * ureg.K).to_base_units()
T_отсчета

<Quantity(854.400+/-200.000, 'kelvin')>

In [47]:
T_экзосферы = (ufloat(1000, 200) * ureg.kelvin).to_base_units()
T_экзосферы

<Quantity(1000.00+/-200.00, 'kelvin')>

Todo: разобраться, почему `ureg` не может сам разобраться с конверсией единиц измерения

In [48]:
k = (k_J_per_K * ureg.J / ureg.K).to_base_units()
k

<Quantity(1.380649e-23, 'kilogram * meter ** 2 / second ** 2 / kelvin')>

In [52]:
p_экз = ureg.k * T_экзосферы /  σ / H
p_экз.to('pascal')

<Quantity((2.87635+/-1.31309)e-06, 'pascal')>

In [57]:
P_отсчета / p_экз

<Quantity((4.09440+/-1.92979)e-22, 'kilogram * meter ** 2 / second ** 2 / boltzmann_constant / kelvin')>

In [56]:
h_экз = H_отсчета + H * math.log(P_отсчета / p_экз)
h_экз

TypeError: can't convert an affine function (<class 'uncertainties.core.AffineScalarFunc'>) to float; use x.nominal_value